In this notebook we will restore a table from its backup stored in avro format.

We will use the suggested code on GCP's official docs

In [9]:
import io
import os
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

project = os.environ['GCP_PROJECT']
dataset_id = 'globant'
table_id = 'jobs'
bucket_name = os.environ['GCP_BUCKET'] + '/backups'
full_table_id = project+'.'+dataset_id+'.'+table_id


In [ ]:
def parse_schema(schema_string):
    schema_list = schema_string.split(',')
    schema = []
    for item in schema_list:
        item_s = item.split(':')
        schema.append(bigquery.SchemaField(item_s[0], item_s[1]))
    return schema
        

In [14]:
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    source_format=bigquery.SourceFormat.AVRO,
)

uri = 'gs://' + bucket_name + '/' + table_id


In [17]:
load_job = client.load_table_from_uri(
    uri, full_table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(full_table_id)
print("Table {} restored with {} rows.".format(table_id, destination_table.num_rows))

Table restored with 198 rows.
